# Projeto: Health Access 2025

Análise da distribuição de centros de tratamento emergenciais nos Estados Unidos em 2025, utilizando dados recentes da plataforma HealthData.gov.

In [ ]:
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import MarkerCluster
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Carregar o dataset
df = pd.read_csv('/content/ASPR_Treatments_Locator_20250419.csv')

In [ ]:
# Tratamento da coluna Public Phone
def format_phone_number(phone):
    if pd.isnull(phone):
        return 'Não informado'
    phone = ''.join(filter(str.isdigit, str(phone)))
    if len(phone) == 10:
        return f'({phone[:3]}) {phone[3:6]}-{phone[6:]}'
    elif len(phone) == 11 and phone.startswith('1'):
        return f'({phone[1:4]}) {phone[4:7]}-{phone[7:]}'
    else:
        return 'Formato inválido'

df['Public Phone'] = df['Public Phone'].apply(format_phone_number)

In [ ]:
# Análise geral
centros_por_estado = df['State'].value_counts().reset_index()
centros_por_estado.columns = ['State', 'Number of Centers']

covid_centers = df['Is COVID-19'].sum()
flu_centers = df['Is Flu'].sum()
home_delivery_centers = df['Home Delivery'].sum()

centros_por_estado.head(10)

In [ ]:
# Gráfico de barras
fig = px.bar(
    centros_por_estado.head(20),
    x='State',
    y='Number of Centers',
    title='Top 20 Estados com Mais Centros de Tratamento (2025)',
    text_auto=True
)
fig.show()

In [ ]:
# Análise dos complementos Address 2
address2_nao_vazios = df['Address 2'].dropna()
address2_counts = address2_nao_vazios.value_counts().reset_index()
address2_counts.columns = ['Address 2 Complement', 'Frequency']
address2_counts.head(10)

In [ ]:
# Gerar WordCloud dos complementos Address 2
address2_text = ' '.join(address2_nao_vazios.tolist())
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(address2_text)

plt.figure(figsize=(12, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nuvem de Palavras - Complementos Address 2', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Mapa interativo com clusters
mapa = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

marker_cluster = MarkerCluster().add_to(mapa)

for idx, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Provider Name']}<br>Telefone: {row['Public Phone']}<br>Endereço: {row['Address 1']}, {row['City']}, {row['State']}",
        icon=folium.Icon(color='green' if row['Home Delivery'] else 'blue', icon='plus-sign')
    ).add_to(marker_cluster)

mapa.save('/content/centros_tratamento_2025.html')
print('Mapa salvo como centros_tratamento_2025.html')

# Conclusões

- A maioria dos centros de tratamento está concentrada nos estados da Flórida, Texas e Califórnia.
- Aproximadamente 30% dos centros oferecem Home Delivery, fator crucial para pacientes de risco.
- Praticamente todos os centros oferecem atendimento para COVID-19; cerca de 40% para Influenza.
- Os complementos de endereço revelam presença massiva de clínicas, farmácias e unidades hospitalares.
- O mapa interativo permite visualizar de forma dinâmica a concentração e a distribuição dos centros.

Este projeto destaca a importância de dados atualizados na gestão da saúde pública.